In [1]:
import Pkg; Pkg.add(Pkg.PackageSpec(url="https://github.com/JuliaComputing/JuliaAcademyData.jl"))
using JuliaAcademyData; activate("Deep learning with Flux")

    Updating git-repo `https://github.com/JuliaComputing/JuliaAcademyData.jl`
    Updating registry at `~/.julia/registries/General`
┌ Error: curl_easy_setopt: 48
└ @ Downloads.Curl /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.6/Downloads/src/Curl/utils.jl:36
┌ Error: curl_easy_setopt: 48
└ @ Downloads.Curl /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.6/Downloads/src/Curl/utils.jl:36
┌ Error: curl_easy_setopt: 48
└ @ Downloads.Curl /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.6/Downloads/src/Curl/utils.jl:36
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.6/Project.toml`
  No Changes to `~/.julia/environments/v1.6/Manifest.toml`
  Activating environment at `~/.julia/packages/JuliaAcademyData/3C2GY/courses/Deep learning with Flux/Project.toml`
Precompiling project...
  ✓ JSON
  ✓ Conda
  ✓ FFTW
  ✓ FFTViews
  ✗ Zygote
  ✓ CategoricalArrays
  ✓ ImageFiltering
  ✓ Ima

In [ ]:
Pkg.precompile()

Precompiling project...
  ✓ JSON
  ✓ Conda
  ✓ FFTW
  ✓ FFTViews
  ✗ Zygote


<br/>

# Intro to Flux.jl

In the previous course, we learned how machine learning allows us to classify data as apples or bananas with a single neuron. However, some of those details are pretty fiddly! Fortunately, Julia has a powerful package that does much of the heavy lifting for us, called [`Flux.jl`](https://fluxml.github.io/).

*Using `Flux` will make classifying data and images much easier!*

## Using `Flux.jl`

We can get started with `Flux.jl` via:

In [3]:
# using Pkg; Pkg.add(["Flux", "Plots"])
using Pkg

Pkg.add("Flux")
Pkg.add("Zygote")



   Resolving package versions...


LoadError: Unsatisfiable requirements detected for package [38;5;14mCuArrays [3a865a2d][39m:
 [38;5;14mCuArrays [3a865a2d][39m log:
 ├─possible versions are: [38;5;14m0.2.1-2.2.2[39m or uninstalled
 ├─restricted by compatibility requirements with [38;5;2mFlux [587475ba][39m to versions: [38;5;14m1.4.3-1.7.3[39m
 │ └─[38;5;2mFlux [587475ba][39m log:
 │   ├─possible versions are: [38;5;2m0.4.1-0.12.8[39m or uninstalled
 │   └─restricted to versions [38;5;2m0.10.1[39m by an explicit requirement, leaving only versions [38;5;2m0.10.1[39m
 └─restricted by julia compatibility requirements to versions: uninstalled — no versions left

In [ ]:
using Plots

#### Helpful built-in functions

When working we'll `Flux`, we'll make use of built-in functionality that we've had to create for ourselves in previous notebooks.

For example, the sigmoid function, σ, that we have been using already lives within `Flux`:

In [5]:
?σ

"σ" can be typed by \sigma<tab>

search:

Couldn't find σ
Perhaps you meant !, %, &, ', *, +, -, /, :, <, >, \, ^, |, ~, ÷, π, ℯ, ∈ or ∉


No documentation found.

Binding `σ` does not exist.


In [6]:
plot(σ, -5, 5, label="\\sigma", xlabel="x", ylabel="\\sigma\\(x\\)")

LoadError: UndefVarError: plot not defined

Importantly, `Flux` allows us to *automatically create neurons* with the **`Dense`** function. For example, in the last notebook, we were looking at a neuron with 2 inputs and 1 output:

 <img src="https://raw.githubusercontent.com/JuliaComputing/JuliaAcademyData.jl/master/courses/Deep%20learning%20with%20Flux/data/single-neuron.png" alt="Drawing" style="width: 500px;"/>

 We could create a neuron with two inputs and one output via

In [ ]:
model = Dense(2, 1, σ)

This `model` object comes with places to store weights and biases:

In [ ]:
model.W

In [ ]:
model.b

In [ ]:
typeof(model.W)

In [ ]:
x = rand(2)
model(x)

In [ ]:
σ.(model.W*x + model.b)

Unlike in previous notebooks, note that `W` is no longer a `Vector` (1D `Array`) and `b` is no longer a number! Both are now stored in so-called `TrackedArray`s and `W` is effectively being treated as a matrix with a single row. We'll see why below.

Other helpful built-in functionality includes ways to automatically calculate gradients and also the cost function that we've used in the previous course -

$$L(w, b) = \sum_i \left[y_i - f(x_i, w, b) \right]^2$$

If you normalize by dividing by the total number of elements, this becomes the "mean square error" function, which in `Flux` is named **`Flux.mse`**.

In [ ]:
methods(Flux.mse)

### Bringing it all together

Load the datasets that contain the features of the apple and banana images.

In [ ]:
using CSV, DataFrames

apples = DataFrame(CSV.File(datapath("data/apples.dat"), delim='\t', allowmissing=:none, normalizenames=true))
bananas = DataFrame(CSV.File(datapath("data/bananas.dat"), delim='\t', allowmissing=:none, normalizenames=true));

In [ ]:
x_apples  = [ [row.red, row.green] for row in eachrow(apples)]
x_bananas = [ [row.red, row.green] for row in eachrow(bananas)];

Concatenate the x (features) together to create a vector of all our datapoints, and create the corresponding vector of known labels:

In [ ]:
xs = [x_apples; x_bananas]
ys = [fill(0, size(x_apples)); fill(1, size(x_bananas))];

In [ ]:
model = Dense(2, 1, σ)

We can evaluate the model (currently initialized with random weights) to see what the output value is for a given input:

In [ ]:
model(xs[1])

And of course we can examine the current loss value for that datapoint:

In [ ]:
loss = Flux.mse(model(xs[1]), ys[1])

In [ ]:
typeof(loss)

### Backpropagation

In [ ]:
model.W

In [ ]:
model.W.grad

In [ ]:
using Flux.Tracker
back!(loss)

In [ ]:
model.W.grad

Now we have all the tools necessary to build a simple gradient descent algorithm!

### The easy way

You don't want to manually write out gradient descent algorithms every time! Flux, of course, also brings in lots of optimizers that can do this all for you.

In [ ]:
?SGD

In [ ]:
?Flux.train!

So we can simply define our loss function, an optimizer, and then call `train!`. That's basic machine learning with Flux.jl.

In [ ]:
model = Dense(2, 1, σ)
L(x,y) = Flux.mse(model(x), y)
opt = SGD(params(model))
Flux.train!(L, zip(xs, ys), opt)

## Visualize the result

In [ ]:
contour(0:.1:1, 0:.1:1, (x, y) -> model([x,y])[].data, fill=true)
scatter!(first.(x_apples), last.(x_apples), label="apples")
scatter!(first.(x_bananas), last.(x_bananas), label="bananas")
xlabel!("mean red value")
ylabel!("mean green value")